In [7]:
!cat /home/chen/chengxu/LSHVec-master/my/traindata/lab*.fq > /home/chen/chengxu/LSHVec-master/my/data.fastq


In [8]:
!python /home/chen/chengxu/LSHVec-master/scripts/fastseq/fastqToSeq.py -i /home/chen/chengxu/LSHVec-master/my/data.fastq -o /home/chen/chengxu/LSHVec-master/my/data.seq


In [9]:
!head -n 3 /home/chen/chengxu/LSHVec-master/my/data.seq

1	label|2|.11861_6_61.1-9598/1	TATGCCGAACAAAGGTACGAAAAATCGGGGCGAAGATACGGTTTTGCGCTTGTCCGGCTGTCGGAAACAGTTGTTTTGCTTTTGGTTTTTATGTCTTAATTGATTGATTTATATAGGTTTATTGTGTGTGTCGAAATGGCGGAGTGAGGG
2	label|2|.11861_6_61.1-9596/1	CCTTTTCGGCCGGTACGGTATCCGCCGTCATGAAAGGCCCGTAATTCTTCTCCATGCGGTGCAGCATGGCTTCCTGTTTGGCTGTTTTGATGCCGAATGTCTCTCCGGCTACCGTTACGTATATCATCGGCATATGTTCTCTATACGTTA
3	label|2|.11861_6_61.1-9594/1	TTCCCCCGCGCAAGGGCAGCAACGATGCCCCGAGCCACATCGCTGGCCGCGACAAAATTTTTGCCACCGGGCGGACAAAACACGATACGACGGCCATACCCCATCAGGATGATGCGTCCCGAACTCGGTCGGGAATCCCATGCACCAATC


In [10]:
!python /home/chen/chengxu/LSHVec-master/scripts/fastseq/hashSeq.py -i /home/chen/chengxu/LSHVec-master/my/data.seq --hash lsh -o /home/chen/chengxu/LSHVec-master/my/data15.fnv -k 15

2023-05-24 20:52:51,401 - hashSeq - INFO - start converting...
2023-05-24 20:52:51,402 - hashSeq - INFO - parameters: {'in_file': '/home/chen/chengxu/LSHVec-master/my/data.seq', 'out_file': '/home/chen/chengxu/LSHVec-master/my/data15.fnv', 'hash_size': 22, 'batch_size': 100000, 'create_lsh_only': False, 'lsh_file': '', 'f': <function convert.<locals>.f at 0x7fe590bc6598>, 'bucket': 20000000, 'hash_fun': 'lsh', 'kmer_size': 15, 'n_thread': 3}
2023-05-24 20:52:51,402 - hashSeq - INFO - creating hash ...
100%|████████████████████████████████████████| 660/660 [00:00<00:00, 843.88it/s]
2023-05-24 20:52:54,053 - hashSeq - INFO - finish creating hash ...
100%|██████████████████████████████████| 100000/100000 [01:56<00:00, 693.17it/s]
2023-05-24 20:54:59,498 - hashSeq - INFO - written 100000 lines
100%|█████████████████████████████████| 100000/100000 [01:29<00:00, 1114.18it/s]
2023-05-24 20:56:37,585 - hashSeq - INFO - written 200000 lines
100%|█████████████████████████████████| 100000/100000 

In [ ]:
import numpy as np
def read_embedding(fname):
    with open(fname) as fin:
        lines=list(fin)
    assert len(lines)>1
    first_line=lines[0]
    num_word, dim_vec = [int(u) for u in first_line.split(" ")]
    assert len(lines)==num_word+1
    ret ={}
    for line in lines[1:]:
        lst = line.strip().split(" ")
        assert len(lst)==dim_vec+1, line
        if lst[0]=='</s>': continue
        word = int(lst[0])
        vec = np.array([float(u) for u in lst[1:]])
        ret[word]=vec
    return ret,dim_vec
embedding,dim_vec = read_embedding("/home/chen/chengxu/LSHVec-master/my/lshvec.vec")
seqvectors = []
with open("/home/chen/chengxu/LSHVec-master/my/data15.fnv") as fin:
    lines = list(fin)
for i,line in enumerate(lines):
    words = [int(u) for u in line.strip().split(" ")]
    vec = [embedding[u] for u in words if u in embedding]
    seqvectors.append(np.mean(vec,0) if len(vec)>0 else None)#np.zeros([dim_vec],dtype=np.float32))

In [2]:
seqvectors[:2]
#vec[:2]

NameError: name 'seqvectors' is not defined

In [1]:
import pandas as pd
seqs = pd.read_csv("/home/chen/chengxu/LSHVec-master/my/data.seq",sep='\t', header=None, usecols=[0,1],index_col=0)
seqs.columns=['id']
seqs['special'] = seqs['id'].apply(lambda x: x.split("|")[1])
seqs['vec']=seqvectors
seqs.drop('id',axis=1,inplace=True)
seqs=seqs[~seqs['vec'].isnull()]
seqs['special'].head(5)

/home/chen/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


NameError: name 'seqvectors' is not defined

In [1]:
import matplotlib.pyplot as plt
from MulticoreTSNE import MulticoreTSNE as TSNE
seqss=seqs.sample(10000,replace=False)
tsne = TSNE(n_components=2, perplexity=100, n_jobs=8)
X=np.array(list(seqss['vec'].values))
X.shape
%time Y = tsne.fit_transform(X)
labels=seqss['special'].values
plt.scatter(Y[labels][:,0],Y[labels][:,1],alpha=1,s=1)
plt.scatter(Y[~labels][:,0],Y[~labels][:,1],alpha=0.5,s=1)
plt.legend(['spike','non-spike'])


NameError: name 'seqs' is not defined